In [2]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Defining global variables
beam_data = None
depth = None
width = None
model = None
similarity_threshold = 0.05

# Function to load beam data from file
def load_beam_data(file_path):
    global beam_data
    beam_data = pd.read_csv(file_path)
    return beam_data

# Function to load collaborative filtering model
def load_collaborative_filtering_model():
    global model, beam_data
    # Determine the rating scale based on the data
    rating_scale_min = beam_data['Plastic Modulus y-y'].min()
    rating_scale_max = beam_data['Plastic Modulus y-y'].max()
    reader = Reader(rating_scale=(rating_scale_min, rating_scale_max))
    # Load data into Surprise's Dataset format
    data = Dataset.load_from_df(beam_data[['Plastic Modulus y-y', 'Beam_Name', 'Plastic Modulus z-z']], reader)
    # Split data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2)
    # Initialize the KNNBasic collaborative filtering model
    model = KNNBasic(sim_options={'name': 'pearson', 'user_based': True, 'shrinkage': 100})
    # Fit the model to the training data
    model.fit(trainset)

# Load beam data from file
load_beam_data('structural-sections_1.csv')

# Load collaborative filtering model
load_collaborative_filtering_model()

# Function to calculate similarity between predictions and calculated values
def calculate_similarity(pred, calculated_value):
    if hasattr(pred, 'est'):  
        return abs(pred.est - calculated_value)
    else:
        return abs(pred - calculated_value)

# Function to extract user input from entry fields
def user_input():
    try:
        applied_force_vertical = float(entry_force_vertical.get())
        applied_force_horizontal = float(entry_force_horizontal.get())
        length = float(entry_length.get())
        global depth, width
        depth = float(entry_depth.get())
        width = float(entry_width.get())
        deflection_limit_value = entry_deflection_limit.get()
        gamma_M = float(entry_gamma_M.get())
        steel_class = float(entry_steel_class.get())

        if deflection_limit_value.upper() == 'Y':
            deflection_limit = float(length * 1000) / 360
        else:
            deflection_limit = float(deflection_limit_value)

        return applied_force_vertical, applied_force_horizontal, length, deflection_limit, gamma_M, steel_class

    except ValueError:
        result_text.set("Error: Please enter valid numeric values.")
        return None

# Function to calculate moments based on applied forces and length
def moment_calc(applied_force_vertical, applied_force_horizontal, length):
    moment_y_y = (1/4) * applied_force_vertical * length
    moment_z_z = (1/4) * applied_force_horizontal * length
    return moment_y_y, moment_z_z

# Function to calculate shear force based on applied vertical force
def shear_force_calc(applied_force_vertical):
    shear_force_z_z = applied_force_vertical / 2
    return shear_force_z_z

# Function to calculate beam deflection
def calculate_deflection(applied_force, length, elastic_modulus, moment_of_inertia):
    deflection = (applied_force * length**3) * 1000 / (48 * elastic_modulus * moment_of_inertia)
    return deflection

# Function to calculate moment resistance of the beam
def moment_resistance_calc(steel_class, gamma_M, applied_force_vertical, applied_force_horizontal, length):
    calculated_moment_yy = (1/4) * applied_force_vertical * length
    calculated_moment_zz = (1/4) * applied_force_horizontal * length

    # Calculate moment resistance using the given formula
    beam_data['Moment Resistance y-y'] = (beam_data['Plastic Modulus y-y'] * steel_class) / gamma_M / 1000
    beam_data['Moment Resistance z-z'] = (beam_data['Plastic Modulus z-z'] * steel_class) / gamma_M / 1000

    # Apply similarity threshold condition
    beam_data.loc[calculate_similarity(beam_data['Moment Resistance y-y'], calculated_moment_yy) > similarity_threshold,
                  'Moment Resistance y-y'] = 0

    beam_data.loc[calculate_similarity(beam_data['Moment Resistance z-z'], calculated_moment_zz) > similarity_threshold,
                  'Moment Resistance z-z'] = 0

# Function to calculate shear area of the beam
def shear_area_calc():
    beam_data['Shear_Area'] = beam_data['Thickness Web'] * beam_data['Depth of section'] - 2 * beam_data['Thickness Flange']

# Function to calculate shear resistance of the beam
def shear_resistance_calc(steel_class, gamma_M, applied_force_vertical):
    calculated_shear_resistance_zz = (beam_data['Shear_Area'] * steel_class) / (3**(1/2) * gamma_M)

    # Apply similarity threshold condition
    if 'Shear Resistance z-z' not in beam_data.columns:
        beam_data['Shear Resistance z-z'] = 0

    beam_data.loc[calculate_similarity(beam_data['Shear Resistance z-z'], calculated_shear_resistance_zz) > similarity_threshold,
                  'Shear Resistance z-z'] = 0

# Function to calculate beam deflections
def calculate_deflections(applied_force_vertical, applied_force_horizontal, length):
    beam_data['Deflection y-y'] = calculate_deflection(applied_force_vertical, length * 1000, 210000,
                                                       beam_data['Second Moment of Area y-y'] * 10000)
    beam_data['Deflection z-z'] = calculate_deflection(applied_force_horizontal, length * 1000, 210000,
                                                       beam_data['Second Moment of Area z-z'] * 10000)


# Function to get recommendations from the collaborative filtering model
def get_recommendations(model, beam_data, **kwargs):

    all_beam_names = list(beam_data['Beam_Name'])

    # Calculate differences between predicted and calculated values
    beam_data['Moment_YY_Diff'] = abs(beam_data['Moment Resistance y-y'] - kwargs.get('moment_yy'))
    beam_data['Moment_ZZ_Diff'] = abs(beam_data['Moment Resistance z-z'] - kwargs.get('moment_zz'))
    beam_data['Shear_ZZ_Diff'] = abs(beam_data['Shear Resistance z-z'] - kwargs.get('shear_force_zz'))

    # Combine differences into a combined score
    beam_data['Combined_Diff'] = (
        beam_data['Moment_YY_Diff'] +
        beam_data['Moment_ZZ_Diff'] +
        beam_data['Shear_ZZ_Diff']
    )

    # Calculate distance from the center for each beam
    beam_data['Distance_From_Center'] = abs(beam_data['Combined_Diff'] - beam_data['Combined_Diff'].mean())

    depth_limit = float(entry_depth.get())  
    width_limit = float(entry_width.get()) 
    deflection_limit_value = float(entry_deflection_limit.get()) 

    # Filter beams based on specified limits
    filtered_beams = beam_data[
        (beam_data['Depth of section'] <= depth_limit) &
        (beam_data['Width of section'] <= width_limit) &
        (beam_data['Deflection y-y'] <= deflection_limit_value)
    ].copy()  

    # Define weights for different parameters
    weight_moment_yy = 1
    weight_moment_zz = 1
    weight_shear_zz = 1
    weight_distance = 1

    # Calculate combined score based on weights
    filtered_beams['Combined_Score'] = (
        weight_moment_yy * filtered_beams['Moment_YY_Diff'] +
        weight_moment_zz * filtered_beams['Moment_ZZ_Diff'] +
        weight_shear_zz * filtered_beams['Shear_ZZ_Diff'] +
        weight_distance * filtered_beams['Distance_From_Center']
    )

    # Sort beams based on combined score
    sorted_beams = filtered_beams.sort_values(by='Combined_Score')
    # Select top 3 recommended beams
    selected_beams = sorted_beams['Beam_Name'].head(3).tolist()
    return selected_beams


# GUI setup
app = tk.Tk()
app.title("Beam Calculator - collaborative filtering model using KNNBasic")

# Colors
bg_color = "tomato"
label_bg_color = "mint cream"
entry_bg_color = "DarkSeaGreen3"
button_bg_color = "ivory2"
button_fg_color = "blue4"

app.configure(bg=bg_color)

# Font settings
font_size = 12
font_style = ('Arial', font_size)

# Labels for input fields
labels_input = ["Enter Applied Vertical Force (kN):", "Enter Applied Horizontal Force (kN):", "Enter Length (m):",
                "Enter Max Depth (mm):", "Enter Max Width (mm):", "Enter Deflection Limit (Y or value):",
                "Enter Partial Safety Factor (gamma_M):", "Enter Steel Class (e.g., 235 for S235 steel):"]

# Create labels and entry fields for input
for i, label_text in enumerate(labels_input):
    ttk.Label(app, text=label_text, background=label_bg_color).grid(row=i, column=2, sticky="e", padx=10, pady=10)

entry_force_vertical = ttk.Entry(app, background=entry_bg_color)
entry_force_horizontal = ttk.Entry(app, background=entry_bg_color)
entry_length = ttk.Entry(app, background=entry_bg_color)
entry_depth = ttk.Entry(app, background=entry_bg_color)
entry_width = ttk.Entry(app, background=entry_bg_color)
entry_deflection_limit = ttk.Entry(app, background=entry_bg_color)
entry_gamma_M = ttk.Entry(app, background=entry_bg_color)
entry_steel_class = ttk.Entry(app, background=entry_bg_color)

entry_force_vertical.grid(row=0, column=3, padx=5, pady=5)
entry_force_horizontal.grid(row=1, column=3, padx=5, pady=5)
entry_length.grid(row=2, column=3, padx=10, pady=10)
entry_depth.grid(row=3, column=3, padx=10, pady=10)
entry_width.grid(row=4, column=3, padx=10, pady=10)
entry_deflection_limit.grid(row=5, column=3, padx=10, pady=10)
entry_gamma_M.grid(row=6, column=3, padx=10, pady=10)
entry_steel_class.grid(row=7, column=3, padx=5, pady=5)

# Label to display results
result_text = tk.StringVar()
result_label = ttk.Label(app, textvariable=result_text, background=bg_color)
result_label.grid(row=len(labels_input) + 1, columnspan=5, pady=20)

# Function to clear entry fields
def clear_entries():
    entry_force_vertical.delete(0, 'end')
    entry_force_horizontal.delete(0, 'end')
    entry_length.delete(0, 'end')
    entry_depth.delete(0, 'end')
    entry_width.delete(0, 'end')
    entry_deflection_limit.delete(0, 'end')
    entry_gamma_M.delete(0, 'end')
    entry_steel_class.delete(0, 'end')
    result_text.set("")

# Button to calculate beam properties
calculate_button = ttk.Button(app, text="Calculate", command=calculate_beam, style='Green.TButton')
calculate_button.grid(row=len(labels_input) + 2, columnspan=5, pady=20)

# Button to clear entry fields
clear_button = ttk.Button(app, text="Clear", command=clear_entries, style='Red.TButton')
clear_button.grid(row=len(labels_input) + 3, columnspan=2, pady=10)

# Style configuration for buttons
style = ttk.Style()
style.configure('Green.TButton', background=button_bg_color, foreground=button_fg_color)
style.configure('Red.TButton', background='red3', foreground=button_fg_color)

# Run the GUI main loop
app.mainloop()


Computing the pearson similarity matrix...
Done computing similarity matrix.
